In [7]:
#imports
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
import numpy as np

In [8]:
#gpu config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

Using device: cuda


In [3]:
#hyperparams
learning_rate = 0.01
batch_size = 128
num_epochs = 30

#load data
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.FashionMNIST(root='data/', train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST(root='data/',train=False, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

100%|██████████| 26421880/26421880 [00:03<00:00, 8438218.77it/s] 


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 140994.84it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 2594670.68it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 21024612.46it/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [4]:
#LeNet5
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5, padding=2)
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.pool1(torch.relu(self.conv1(x)))
        x = self.pool2(torch.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [5]:
#initialize
model = LeNet5().to(device)
writer = SummaryWriter()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [6]:
#Training

#train net
for epoch in range(num_epochs):
    train_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        
        # Send data to GPU
        images, labels = images.to(device), labels.to(device)

        # Forward pass and compute loss
        outputs = model(images)
        loss = nn.CrossEntropyLoss()(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Compute running train loss
        train_loss += loss.item()
        
    #validation
    model.eval()
    valid_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            valid_loss += loss.item()*data.size(0)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    
    valid_loss = valid_loss/len(test_loader.dataset)
    accuracy = 100 * correct / total

    writer.add_scalar('Training Loss', train_loss / len(train_loader), epoch)
    writer.add_scalar('Validation Loss', valid_loss / len(test_loader), epoch)
    
    print(f"Epoch {epoch+1}/{30}, Training Loss: {train_loss:.6f}, Validation Loss: {valid_loss:.6f}, Accuracy: {accuracy:.2f}")
writer.close()

Epoch 1/30, Training Loss: 1075.513860, Validation Loss: 2.269834, Accuracy: 24.99
Epoch 2/30, Training Loss: 760.355628, Validation Loss: 0.963715, Accuracy: 62.62
Epoch 3/30, Training Loss: 407.228582, Validation Loss: 0.777832, Accuracy: 69.05
Epoch 4/30, Training Loss: 357.480478, Validation Loss: 0.729541, Accuracy: 72.94
Epoch 5/30, Training Loss: 327.002951, Validation Loss: 0.670212, Accuracy: 74.95
Epoch 6/30, Training Loss: 307.699865, Validation Loss: 0.693618, Accuracy: 74.62
Epoch 7/30, Training Loss: 290.995409, Validation Loss: 0.616184, Accuracy: 77.10
Epoch 8/30, Training Loss: 275.131145, Validation Loss: 0.662803, Accuracy: 75.40
Epoch 9/30, Training Loss: 261.630094, Validation Loss: 0.553625, Accuracy: 79.44
Epoch 10/30, Training Loss: 249.522158, Validation Loss: 0.547330, Accuracy: 79.94
Epoch 11/30, Training Loss: 239.365667, Validation Loss: 0.519575, Accuracy: 80.62
Epoch 12/30, Training Loss: 230.586167, Validation Loss: 0.528089, Accuracy: 79.92
Epoch 13/30,